## Урок 3. Классификация. Логистическая регрессия.

### Домашнее задание

In [36]:
import numpy as np
import matplotlib.pyplot as plt

In [37]:
X = np.array([ [   1,    1,  500,    1],
               [   1,    1,  700,    1],
               [   1,    2,  750,    2],
               [   1,    5,  600,    1],
               [   1,    3, 1450,    2],
               [   1,    0,  800,    1],
               [   1,    5, 1500,    3],
               [   1,   10, 2000,    3],
               [   1,    1,  450,    1],
               [   1,    2, 1000,    2]], dtype=np.float64)

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype=np.float64)

In [38]:
def standard_scale(x):
    res = (x - x.mean()) / x.std()
    return res

In [39]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [40]:
X_st = X.copy()
X_st[:, 2] = standard_scale(X[:, 2])

In [41]:
import warnings
warnings.filterwarnings('ignore')

1. *Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.

In [42]:
def calc_logloss(y, y_pred, eps = 1e-9):
    y_pred_new = y_pred.copy().astype(np.float64)
    y_pred_new[y_pred_new==1] = 1 - 1e-9
    y_pred_new[y_pred_new<=eps] = 1e-9
    err = - np.mean(y * np.log(y_pred_new) + (1.0 - y) * np.log(1.0 - y_pred_new))
    return err

In [43]:
# Пример применения
y1 = np.array([1, 0])
y_pred1 = np.array([0.8, 0.1])
calc_logloss(y1, y_pred1)

0.164252033486018

In [44]:
# Пример c ошибкой
y1 = np.array([1, 0])
y_pred1 = np.array([1, 0.1])
calc_logloss(y1, y_pred1)

0.05268025832891313

2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.


In [45]:
np.random.seed(42)

def eval_model(X, y, iterations, eta):
    W = np.random.randn(X.shape[1])
    n = X.shape[0]
    
    for i in range(iterations):
        z = np.dot(X, W)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        
        dQ = 1/n * X.T @ (y_pred - y)
        W -= eta * dQ
    return [eta, iterations, W, err]

In [46]:
etas = np.logspace(-5, 1, 7)
iterations = np.arange(1000, 20000, 2000)

In [47]:
etas, iterations

(array([1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01]),
 array([ 1000,  3000,  5000,  7000,  9000, 11000, 13000, 15000, 17000,
        19000]))

In [48]:
results = []
for eta in etas:
    for iteration in iterations:
        results.append(eval_model(X_st, y, iteration, eta))

In [49]:
min = [None, None, None, np.inf]
for result in results:
    if result[3] < min[3]:
        min = result
        #print(result)

In [50]:
 min

[10.0,
 19000,
 array([-129.23015537,   -9.91604201,  -37.6084486 ,  100.40265065]),
 0.02770144878359389]

In [51]:
W = min[2]
W

array([-129.23015537,   -9.91604201,  -37.6084486 ,  100.40265065])

**Ответ:** *eta = 10.0, iterations = 19000*

3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).


In [52]:
def calc_pred_proba(W, X):
    y_pred_proba = sigmoid(np.dot(W, X.T))
    return y_pred_proba

In [53]:
y_pred_proba = calc_pred_proba(W, X_st)
y_pred_proba

array([1.29803391e-01, 2.73476706e-08, 1.00000000e+00, 3.79634074e-22,
       9.93214586e-01, 2.37152895e-07, 1.00000000e+00, 1.25257985e-03,
       8.78177728e-01, 1.00000000e+00])

4. Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [54]:
def calc_pred(W, X, treshold=0.5):
    y_pred = sigmoid(np.dot(W, X.T)) > treshold
    return y_pred.astype(int)

In [55]:
y_pred = calc_pred(W, X_st)
y_pred

array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1])

**Вывод:** *удалось добиться вероятностей между 0 и 1. Метод верен.*

5. *Реализуйте функции для подсчета Accuracy, матрицы ошибок, точности и полноты, а также F1 score.

In [56]:
def calc_accuracy(y, y_pred):
    total_successes = sum(y_pred==y)
    return total_successes/len(y)

calc_accuracy(y, y_pred)

1.0

In [57]:
def calc_matrix(y, y_pred):
    pairs = zip(y, y_pred)
    
    true_positive = 0
    true_negative = 0
    false_positive = 0
    false_negative = 0
    
    for origin, pred in pairs:
        if pred and origin:
            true_positive+= 1
        elif pred and not origin:
            false_negative += 1
        elif origin and not pred:
            false_positive += 1
        else:
            true_negative +=1

    return true_positive, true_negative, false_positive, false_negative

In [58]:
def calc_recall(matrix):
    true_positive = matrix[0]
    false_negative = matrix[3]

    return true_positive / (true_positive + false_negative)

def calc_precision(matrix):
    true_positive = matrix[0]
    false_positive = matrix[2]

    return true_positive / (true_positive + false_positive)

def calc_f_score(precision, recall):
    return 2 * recall * precision  / (recall + precision)

In [59]:
matrix = calc_matrix(y, y_pred)
matrix

(5, 5, 0, 0)

In [60]:
recall = calc_recall(matrix)
recall

1.0

In [61]:
precision = calc_precision(matrix)
precision

1.0

In [62]:
calc_f_score(precision, recall)

1.0

6. Могла ли модель переобучиться? Почему?

**Ответ:** *могла! - очень мало данных. Модель становится слишком сложной, чтобы обобщить тренировочные данные и начинает "запоминать" их вместо извлечения общих закономерностей.*